In [ ]:
# Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import chi2_contingency
import seaborn as sns

In [ ]:
#creating a DF out of the entire csv
births_df = pd.read_csv("2021_Data.csv")
births_df.head()

# Data Cleaning: (Emily)

In [ ]:
#choosing columns i want to examine more
clean_births_df = births_df[["dob_yy", "dob_mm","bfacil", "mager", "bmi_r", "priorlive", "priordead", "precare", "previs", "ld_anes", "attend","mm_aicu", "ab_nicu", "pay", "dbwt"]]
clean_births_df.head()

In [ ]:
#rename columns 
clean_births_df = clean_births_df.rename(columns={"dob_yy":"Birth Year", "dob_mm":"Birth Month", 
                                                  "bfacil":"Birth Place", "mager":"Mother's Age", "bmi_r":"Mother's BMI",
                                                  "priorlive":"Living Children", "priordead":"Deceased Children",
                                                 "precare": "Prenatal Care", "previs": "# of Prenatal Visits",
                                                  "ld_anes": "Used Anesthesia", "attend":"Attendant at Birth",
                                                  "mm_aicu":"Intensive Care Admission", "ab_nicu": "NICU Admission", 
                                                  "pay":"Payment Source", "dbwt":"Birth Weight",
                                                 })
clean_births_df.head()

In [ ]:
#Filtering out any data that is unknown (this is represented by the values 9 and/or 99)
clean_births_df = clean_births_df[~clean_births_df.apply(lambda row: any(x in [9, 99] for x in row), axis=1)]

In [ ]:
clean_births_df.dtypes

In [ ]:
#declaring recoding responses as objects to later replace with coded meaning
clean_births_df = clean_births_df.astype({"Birth Place":str}, errors='raise')
clean_births_df = clean_births_df.astype({"Mother's BMI":str}, errors='raise')
clean_births_df = clean_births_df.astype({"Attendant at Birth": str}, errors='raise')
clean_births_df = clean_births_df.astype({"Payment Source": str}, errors='raise')

In [ ]:
#verify datatypes were changed
clean_births_df.dtypes

In [ ]:
#decoding inputs based on documentation
#OK if returns a warning. 
#If returns an error- restart and run all

##For column: Birth place
clean_births_df["Birth Place"] = clean_births_df["Birth Place"].replace({"1":"Hospital", "2": "Freestanding Birth Center",
                                                                         "3":"Home (intended)", "4": "Home (unintended)",
                                                                        "5": "Home (intent unknown)", "6": "Clinic/Doctor's Office",
                                                                        "7":"Other",
                                                                        })
##For column: Mother's BMI
clean_births_df["Mother's BMI"] = clean_births_df["Mother's BMI"].replace({"1":"Underweight <18.5", "2":"Normal 18.5-24.9",
                                                                          "3": "Overweight 25.0-29.9", "4":"Obesity I 35.0-34.9",
                                                                          "5":"Obesity II 35.0-39.9", "6": "Extreme Obesity III ≥ 40.0",
                                                                          })
##For column: Prenatal Care
clean_births_df.loc[(clean_births_df['Prenatal Care'] >= 1) & (clean_births_df['Prenatal Care'] <= 10), 'Prenatal Care'] = 'Y'
clean_births_df.loc[(clean_births_df['Prenatal Care'] == 0), 'Prenatal Care'] = 'N'


##For column: Attendant at Birth
clean_births_df["Attendant at Birth"] = clean_births_df["Attendant at Birth"].replace({"1": "Doctor of Medicine (MD)", "2": "Doctor of Osteopathy (DO)",
                                                                                      "3":"Certified Nurse Midwife/Certified Midwife (CNM/CM)", "4":"Other Midwife",
                                                                                      "5": "Other",
                                                                                      })
##For column: Payment Source
clean_births_df["Payment Source"] = clean_births_df["Payment Source"].replace({"1":"Medicaid", "2":"Private Insurance",
                                                                              "3": "Self-Pay", "4":"Indian Health Service",
                                                                              "5":"CHAMPUS/TRICARE", "6":"Other Government (Federal, State, Local)",
                                                                              "8":"Other"})

#Show updated dataframe
clean_births_df.head()

In [ ]:
#convert baby birth weight from grams to pounds

# Conversion factor from grams to pounds
grams_to_pounds_conversion_factor = 0.00220462

clean_births_df["Birth Weight"] = clean_births_df["Birth Weight"] * grams_to_pounds_conversion_factor

#Rename column 
clean_births_df = clean_births_df.rename(columns={"Birth Weight":"Birth Weight (lbs)"})

#display 
clean_births_df.head()

### Baby Birth Weight Analysis: (Emily)

In [ ]:
#create simplified dataframe

weight_df = clean_births_df[["Birth Weight (lbs)","Payment Source"]]
weight_df.head()

In [ ]:
#name variables i want to compare
weight = weight_df["Birth Weight (lbs)"]
payment = weight_df["Payment Source"]

In [ ]:
#test out what scatter plot looks like
plt.scatter(weight, payment)